# Ordinal Forest SOA Model Replication from Paper

## Prep Data

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
import warnings
warnings.filterwarnings('ignore')


X = pd.read_csv("predictor_df.csv", parse_dates=['index'])
X.rename(columns={'index': 'Quarter'}, inplace=True)

X['Quarter'] = X['Quarter'].dt.to_period('Q')
# last row isn't needed, uncessary
X = X.iloc[:-1, :]

y = pd.read_csv("fomc_meetings.csv", parse_dates=['Date'])  # meeting dates + decision

# convert date to quarter accroding to apper
y['Quarter'] = y['Date'].dt.to_period('Q')
df = y.merge(X, on='Quarter', how='left')

df.tail()

,Date,Outcome,Quarter,CPIAUCSL_QoQ,CPILFESL_QoQ,PPIACO_QoQ,CL=F_QoQ,GDP_QoQ,GDPC1_QoQ,INDPRO_QoQ,...,TB6SMFFM,T10Y2Y,T2Y3M,T5Y2Y,prev1_decision,prev2_decision,unemp_idx,b_idx,ir_idx,pmi_QoQ
250,2024-09-18,CUT,2024Q3,0.549291,0.782803,-1.262924,-16.396864,1.234461,0.759510,-0.637267,...,-0.71,0.15,-1.30,-0.12,5.33,5.33,101.0,79.0,122.0,-1.656315
251,2024-11-07,CUT,2024Q4,0.874064,0.767061,0.297607,5.207574,1.187918,0.607065,0.509087,...,-0.29,0.33,-0.16,0.02,4.64,4.83,99.0,85.0,121.0,3.578947
252,2024-12-18,CUT,2024Q4,0.874064,0.767061,0.297607,5.207574,1.187918,0.607065,0.509087,...,-0.29,0.33,-0.16,0.02,4.64,4.83,99.0,85.0,121.0,3.578947
253,2025-01-29,NO CHANGE,2025Q1,0.633495,0.730909,2.131916,-0.334632,0.000000,0.000000,0.748174,...,-0.23,0.34,-0.37,0.07,4.33,4.33,86.0,75.0,103.0,-0.406504
254,2025-03-19,NO CHANGE,2025Q1,0.633495,0.730909,2.131916,-0.334632,0.000000,0.000000,0.748174,...,-0.23,0.34,-0.37,0.07,4.33,4.33,86.0,75.0,103.0,-0.406504


In [2]:
# label for ordinal
label_map = {'CUT': 0, 'NO CHANGE': 1, 'HIKE': 2}
df['target'] = df['Outcome'].map(label_map)
df[['Quarter', 'target']].tail()

,Quarter,target
250,2024Q3,0
251,2024Q4,0
252,2024Q4,0
253,2025Q1,1
254,2025Q1,1


In [3]:
exclude_cols = ['Date', 'Quarter', 'Decision', 'target', 'Outcome']
feature_cols = [col for col in df.columns if col not in exclude_cols]
df = df.sort_values('Date')

## Ordinal Forest Model with Recursive Training Data for One-step-ahead Forecasts

In [4]:
# !pip install orf
import orf

# CHANGE THIS if you want to predict on more observations
start_date = pd.Timestamp('2018-01-01')
test_mask = df['Date'] >= start_date

train_df = df[~test_mask]
test_df = df[test_mask]

print(f"Training on data before January 2018: {len(train_df)} observations")
print(f"Testing on data from January 2018 onwards: {len(test_df)} observations")


y_true = []
y_pred = []

# have to recursively fit the model as we use all past observations before the meeitng to predict
for i, (_, row) in enumerate(test_df.iterrows()):
    current_date = row['Date']
    print(f"Forecasting for observation {i+1}/{len(test_df)}, date: {current_date}")
    
    one_month_before = current_date - pd.DateOffset(months=1)
    
    train_mask = df['Date'] <= one_month_before
    X_train = df[train_mask][feature_cols]
    y_train = df[train_mask]['target']
    
    X_test = row[feature_cols].to_frame().T
    y_test = row['target']
    
    # THIS MAY NOT BE THE RIGHT CHOICE BUT ENDED UP JSUT IMPUTING AS THE STOCK DATA IS MISSING SOME OBSERVATIONS
    X_train = X_train.fillna(method='ffill').fillna(X_train.mean())
    X_test = X_test.fillna(X_train.mean())  
    
    # PARAMAERS MAY DIFFER FROM THE PAPER
    oforest = orf.OrderedForest(
        n_estimators=1000,
        min_samples_leaf=5,
        max_features=3,
        replace=False,
        sample_fraction=0.8,
        honesty=True,
        honesty_fraction=0.5,
        inference=False,
        n_jobs=-1,
        random_state=42
    )
    
    oforest.fit(X=X_train, y=y_train)
    

    pred_probs = oforest.predict(X=X_test, prob=True)
    pred = pred_probs['predictions'].argmax(axis=1)[0]
    
    y_true.append(y_test)
    y_pred.append(pred)
    
    print(f"Actual: {list(label_map.keys())[y_test]}, Predicted: {list(label_map.keys())[pred]}")
    print(f"Training data size: {len(X_train)} observations")
    print("-" * 50)

label_inverse = {0: 'CUT', 1: 'NO CHANGE', 2: 'HIKE'}
y_true_labels = [label_inverse[y] for y in y_true]
y_pred_labels = [label_inverse[y] for y in y_pred]

print("\n=== Classification Report ===")
print(classification_report(y_true, y_pred, target_names=["Cut", "No change", "Hike"]))

print("\n=== Confusion Matrix ===")
conf_matrix = confusion_matrix(y_true, y_pred)
print(conf_matrix)


Training on data before January 2018: 196 observations
Testing on data from January 2018 onwards: 59 observations
Forecasting for observation 1/59, date: 2018-01-31 00:00:00
Actual: NO CHANGE, Predicted: HIKE
Training data size: 196 observations
--------------------------------------------------
Forecasting for observation 2/59, date: 2018-03-21 00:00:00
Actual: HIKE, Predicted: NO CHANGE
Training data size: 197 observations
--------------------------------------------------
Forecasting for observation 3/59, date: 2018-05-02 00:00:00
Actual: NO CHANGE, Predicted: NO CHANGE
Training data size: 198 observations
--------------------------------------------------
Forecasting for observation 4/59, date: 2018-06-13 00:00:00
Actual: HIKE, Predicted: NO CHANGE
Training data size: 199 observations
--------------------------------------------------
Forecasting for observation 5/59, date: 2018-08-01 00:00:00
Actual: NO CHANGE, Predicted: NO CHANGE
Training data size: 200 observations
------------

NameError: name 'np' is not defined

In [9]:
import numpy as np

class_correct = np.diag(conf_matrix)
class_total = np.sum(conf_matrix, axis=1)
class_accuracy = class_correct / class_total * 100

print("\n=== Accuracy by Class ===")
for i, label in enumerate(["Cut", "No change", "Hike"]):
    if class_total[i] > 0:  
        print(f"{label}: {class_accuracy[i]:.1f}%")
    # shouldn't happen
    else:
        print(f"{label}: N/A (no observations)")

overall_accuracy = np.sum(class_correct) / np.sum(class_total) * 100
print(f"Overall Accuracy: {overall_accuracy:.1f}%")

# export
results_df = pd.DataFrame({
    'Date': test_df['Date'].reset_index(drop=True),
    'Actual': y_true_labels,
    'Predicted': y_pred_labels,
    'Correct': [a == p for a, p in zip(y_true_labels, y_pred_labels)]
})


=== Accuracy by Class ===
Cut: 0.0%
No change: 94.4%
Hike: 53.3%
Overall Accuracy: 71.2%


In [10]:
results_df.to_csv("fomc_forecast_results_2018onwards.csv", index=False)

## Differences in Implementation

1. Software implementation: The original paper uses the R package "ordinalForest" while our implementation uses a Python package. This may result in some algorithmic differences despite similar conceptual approaches.
2. Time period: The original paper examines data from January 1994 to June 2022, while our implementation starts predictions from January 2018 onwards.
3. Parameter selection: The paper uses default parameters from the R implementation, while our Python implementation uses similar but not identical parameters.
4. Feature selection: The paper carefully constructs 45 specific predictor variables with both year-over-year and quarter-over-quarter transformations. Our implementation uses only tests quater-over-quarter transformations.
5. Evaluation metrics: We've implemented the same evaluation approach as the paper, looking at class-specific and overall accuracy.
6. Fixed window vs Increasing Window: The paper tested both approaches but we only did the increasing window.